In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r"/Users/loganheydt/Documents/GitHub/Kaggle_competitions/Wine_Quality/data/train.csv")

In [4]:
df.head()

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,6.9,0.725,0.00,2.4,0.093,14.0,31.0,0.99640,3.48,0.54,9.8,5.0
1,1,7.2,0.610,0.06,1.6,0.084,6.0,14.0,0.99765,3.53,0.75,10.9,7.0
2,2,9.3,0.310,0.34,2.7,0.084,32.0,71.0,0.99760,3.25,0.87,9.3,7.0
3,3,6.4,0.440,0.09,2.5,0.082,4.0,12.0,0.99498,3.32,0.84,9.8,6.0
4,4,7.8,0.540,0.00,1.8,0.080,15.0,37.0,0.99720,3.52,0.56,9.8,5.0
